In [1]:
import scvi

D:\Anaconda\envs\scvi_test\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(scvi.__version__)

1.1.1


In [3]:
import jaxlib
jaxlib.__version__

'0.4.23'

In [4]:
import scanpy as sc
from sklearn.model_selection import train_test_split

In [5]:
import pandas as pd

In [6]:
adata=sc.read('D:\\TCGA\mbVDiT_data\\WXS\\blood\\stad\\microdata.h5ad')
sc.pp.normalize_total(adata, target_sum=1e5)
sc.pp.log1p(adata)
origin_data = adata.X
#origin_data, temp_data = train_test_split(origin_data, test_size=0.2, random_state=3407)
origin_data.shape

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\DELL\AppData\Local\Temp\ipykernel_13136\422678120.py:1: SyntaxWarning: invalid escape sequence '\m'
  adata=sc.read('D:\\TCGA\mbVDiT_data\\WXS\\blood\\coad\\microdata.h5ad')
D:\Anaconda\envs\scvi_test\Lib\site-packages\anndata\__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
D:\Anaconda\envs\scvi_test\Lib\site-packages\scanpy\preprocessing\_normalization.py:196: UserWarning: Some cells have zero counts
  warn(UserWarning('Some cells have zero counts'))


(410, 106)

In [7]:
def mask_tensor_with_masks(X, mask_zero_ratio, mask_nonzero_ratio, decice = 'cuda:0'):

    nonzero_mask = np.zeros((X.shape[0], X.shape[1]))
    zero_mask = np.zeros((X.shape[0], X.shape[1]))

    nonzero_indices = np.transpose(np.nonzero(X.cpu()))
    num_indices_to_choose = int(mask_nonzero_ratio * len(nonzero_indices[0]))
    # chosen_indices = np.random.choice(num_indices_to_choose, size=num_indices_to_choose, replace=False)
    chosen_indices_row = np.random.choice(len(nonzero_indices[0]), size=num_indices_to_choose, replace=False)
    #chosen_indices_col = np.random.choice(nonzero_indices[1], size=num_indices_to_choose, replace=False)

    for idx in chosen_indices_row:
        row, col = nonzero_indices[0][idx], nonzero_indices[1][idx]
        nonzero_mask[row, col] = 1

    zero_indices = np.where(X.cpu() == 0)
    num_indices_to_choose = int(mask_zero_ratio * len(zero_indices[0]))
    # chosen_indices = np.random.choice(num_indices_to_choose, size=num_indices_to_choose, replace=False)

    chosen_indices_row = np.random.choice(len(zero_indices[0]), size=num_indices_to_choose, replace=False)
    #chosen_indices_col = np.random.choice(zero_indices[1], size=num_indices_to_choose, replace=False)

    for idx in chosen_indices_row:
        row, col = zero_indices[0][idx], zero_indices[1][idx]
        zero_mask[row, col] = 1

    print()
    # mask = np.bitwise_or(nonzero_mask.astype(int), zero_mask.astype(int))
    # mask_X = X.cpu() * (1 - nonzero_mask)

    all_mask = np.array(nonzero_mask)
    mask = all_mask == 1
    org_data = np.array(X.cpu())[mask]

    mask_X = X.cpu() * (1 - nonzero_mask)

    return mask_X, nonzero_mask, zero_mask, org_data

In [8]:
import torch
x = torch.tensor(origin_data)
x.shape

torch.Size([410, 106])

In [9]:
import numpy as np
mask_x, nonzero_mask, zero_mask, org_data = mask_tensor_with_masks(x, 0, 0.3)

In [10]:
from anndata import AnnData
adata = AnnData(X=np.array(mask_x))
adata.X

array([[11.03992911,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 9.21935003,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [11]:
#adata.layers["counts"] = adata.X.copy()
#adata.layers["counts"] = mask_x.copy()
adata.layers["counts"] = adata.X.copy()
adata.layers["counts"] = np.array(mask_x).copy()

In [12]:
if 'counts' in adata.layers:
    print("Layer 'counts' 存在")
else:
    print("Layer 'counts' 不存在")

Layer 'counts' 存在


In [13]:
# 计算数据的条件数
condition_number = np.linalg.cond(adata.X)
print(f"Condition number: {condition_number}")

# 如果条件数非常大，可能需要进一步处理数据
if condition_number > 1e10:
    print("警告：数据的条件数非常大，可能存在数值不稳定性")

Condition number: 7.501996498272304e+65
警告：数据的条件数非常大，可能存在数值不稳定性


In [14]:
if np.any(np.isnan(adata.X)):
    print("警告：数据中存在NaN值")

In [15]:
adata.raw = adata  # keep full dimension safe
sc.pp.highly_variable_genes(
    adata,
    flavor="seurat_v3",
    n_top_genes=200,
    layer="counts",
    subset=True
)
# sc.tl.pca(adata)
# sc.pp.neighbors(adata)
# sc.tl.umap(adata)
# sc.pl.umap(adata,color=["celltype","BATCH"])

D:\Anaconda\envs\scvi_test\Lib\site-packages\scanpy\preprocessing\_highly_variable_genes.py:64: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(


In [16]:
adata.layers["counts"] = adata.X.copy()
scvi.model.SCVI.setup_anndata(adata, layer="counts")

<frozen abc>:119: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.

D:\Anaconda\envs\scvi_test\Lib\site-packages\scvi\data\fields\_base_field.py:64: UserWarning: adata.layers[counts] does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


In [17]:
vae = scvi.model.SCVI(adata)
vae

D:\Anaconda\envs\scvi_test\Lib\site-packages\scvi\model\_scvi.py:155: UserWarning: This dataset has some empty cells, this might fail inference.Data should be filtered with `scanpy.pp.filter_cells()`
  library_log_means, library_log_vars = _init_library_size(


SCVI model with the following parameters: 
n_hidden: 128, n_latent: 10, n_layers: 1, dropout_rate: 0.1, dispersion: gene, gene_likelihood: zinb, 
latent_distribution: normal.
Training status: Not Trained
Model's adata is minified?: False

In [18]:
vae.train()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
D:\Anaconda\envs\scvi_test\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.
D:\Anaconda\envs\scvi_test\Lib\site-packages\lightning\pytorch\loops\fit_loop.py:293: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 5/400:   1%|▏                | 5/400 [00:00<00:22, 17.73it/s, v_num=1, train_loss_step=27, train_loss_epoch=24.5]

D:\Anaconda\envs\scvi_test\Lib\site-packages\scvi\module\_vae.py:458: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)
D:\Anaconda\envs\scvi_test\Lib\site-packages\scvi\module\_vae.py:458: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)
D:\Anaconda\envs\scvi_test\Lib\site-packages\scvi\module\_vae.py:458: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)
D:\Anaconda\envs\scvi_test\Lib\site-packages\scvi\module\_vae.py:458: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)
D:\Anaconda\envs\scvi_test\Lib\site-packages\scvi\module\_vae.py:458: UserWarning: The value argument must be within the support of the distribution
  r

Epoch 7/400:   2%|▎              | 7/400 [00:00<00:13, 29.38it/s, v_num=1, train_loss_step=20.1, train_loss_epoch=22.4]

D:\Anaconda\envs\scvi_test\Lib\site-packages\scvi\module\_vae.py:458: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)
D:\Anaconda\envs\scvi_test\Lib\site-packages\scvi\module\_vae.py:458: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)
D:\Anaconda\envs\scvi_test\Lib\site-packages\scvi\module\_vae.py:458: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)


...

Epoch 399/400: 100%|██████████▉| 399/400 [00:10<00:00, 38.72it/s, v_num=1, train_loss_step=14.7, train_loss_epoch=13.3]

D:\Anaconda\envs\scvi_test\Lib\site-packages\scvi\module\_vae.py:458: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)
D:\Anaconda\envs\scvi_test\Lib\site-packages\scvi\module\_vae.py:458: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)
D:\Anaconda\envs\scvi_test\Lib\site-packages\scvi\module\_vae.py:458: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)
D:\Anaconda\envs\scvi_test\Lib\site-packages\scvi\module\_vae.py:458: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)
D:\Anaconda\envs\scvi_test\Lib\site-packages\scvi\module\_vae.py:458: UserWarning: The value argument must be within the support of the distribution
  r

Epoch 400/400: 100%|███████████| 400/400 [00:10<00:00, 38.72it/s, v_num=1, train_loss_step=13.7, train_loss_epoch=13.3]

D:\Anaconda\envs\scvi_test\Lib\site-packages\scvi\module\_vae.py:458: UserWarning: The value argument must be within the support of the distribution
  reconst_loss = -generative_outputs["px"].log_prob(x).sum(-1)
`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|███████████| 400/400 [00:10<00:00, 37.67it/s, v_num=1, train_loss_step=13.7, train_loss_epoch=13.3]


In [19]:
adata.obsm["X_scVI"] = vae.get_latent_representation()

In [20]:
SCVI_LATENT_KEY = "X_scVI"

latent = vae.get_latent_representation()
adata.obsm[SCVI_LATENT_KEY] = latent
latent.shape

(410, 10)

In [21]:
denoised = vae.get_normalized_expression(adata, library_size=1e4)
denoised.shape

(410, 106)

In [22]:
denoised

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,105
0,1702.685791,0.978968,0.244076,0.012513,95.961311,76.657700,0.002454,97.341248,11.862943,0.063085,...,0.003743,0.028535,0.175610,0.002701,0.004087,0.009442,0.141615,0.046014,7.813401,0.000320
1,2566.170898,3.302536,0.039259,0.040902,24.740671,11.250848,0.822263,448.995148,0.484526,2.374751,...,0.046973,0.020768,0.032353,0.058967,0.022135,0.289352,1.575131,2.131968,3.450783,0.035387
2,1001.431335,19.417608,3.297479,2.080817,38.458225,41.898029,37.940521,91.326416,10.129892,5.769340,...,3.493703,2.032006,3.054575,2.527580,2.436892,3.938260,102.170700,9.249150,14.327413,3.498808
3,746.091919,10.388915,2.743108,3.358701,460.435791,84.601715,4.070991,76.087097,6.603870,6.868962,...,1.944075,0.275828,0.378870,0.560899,0.876278,0.601052,18.517523,13.592501,27.768326,1.727410
4,2246.720947,33.103138,0.129197,0.102090,16.954773,63.120842,2.530597,269.549683,0.429608,0.043806,...,0.047271,0.067413,0.125030,0.062153,0.037549,0.018097,11.803867,0.048919,12.591056,0.041312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,1111.201294,2.375840,0.117298,0.291367,2.540542,8.341571,7.701284,19.675465,0.384054,0.038858,...,0.189767,0.023321,0.077986,0.114181,0.079088,0.136080,1340.275757,0.031372,0.909478,0.075773
406,1093.928711,30.908592,7.390895,4.472696,380.951721,268.474884,5.851510,462.363556,46.841675,4.414210,...,3.766765,1.473979,2.052898,3.039347,3.245087,1.815966,27.089142,3.699843,139.633942,1.543662
407,511.951508,6.487950,1.008579,0.038034,844.170837,714.587524,0.066098,146.063477,10.495024,0.287135,...,0.025833,0.562429,1.174548,0.054952,0.038135,0.050367,0.548938,0.055491,631.853882,0.009002
408,1000.505249,113.344955,2.114142,36.627228,150.911865,66.287323,3.026417,35.892666,16.383398,402.141968,...,2.154584,3.856749,4.755974,1.392886,1.615846,11.304996,16.827240,380.811279,6.327890,3.683639


In [23]:
adata.X.shape

(410, 106)

In [24]:
import pandas as pd
import numpy as np
pd.DataFrame(np.array(adata.X))

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,105
0,11.039929,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,10.697250,0.0,0.0,0.0,8.803836,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,11.322634,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.086479,0.0,0.0,0.0
406,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
407,0.000000,0.0,0.0,0.0,8.016748,8.016748,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
408,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [25]:
import numpy as np

In [26]:
impute = np.array(denoised)
impute.shape

(410, 106)

In [27]:
org_data.shape

(464,)

In [28]:
all_mask = np.array(nonzero_mask)
mask = all_mask == 1
impute_data = impute[mask]
impute_data.shape

(464,)

In [29]:
from scipy.stats import pearsonr
pearsonr(impute_data, org_data)[0]

0.5153277093705242

In [30]:
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

def rmse(a, b):
    mse = np.mean((a - b) ** 2)
    rmse = np.sqrt(mse)
    return rmse

def mae(a, b):
    mae = np.mean(np.abs(a - b))
    return mae

In [31]:
cosine_similarity(impute_data, org_data)

0.7476771879847866

In [32]:
prediction_result_final = origin_data + impute * (1 - nonzero_mask)
pred_result = pd.DataFrame(prediction_result_final)
pred_result

,0,1,2,3,4,5,6,7,8,9,...,96,97,98,99,100,101,102,103,104,105
0,1713.725720,0.978968,0.244076,0.012513,95.961311,76.657700,0.002454,97.341248,11.862943,0.063085,...,0.003743,0.028535,0.175610,0.002701,0.004087,0.009442,0.141615,0.046014,7.813401,0.000320
1,2566.170898,3.302536,0.039259,0.040902,24.740671,11.250848,0.822263,448.995148,0.484526,2.374751,...,0.046973,0.020768,0.032353,0.058967,0.022135,0.289352,1.575131,2.131968,3.450783,0.035387
2,1001.431335,19.417608,3.297479,2.080817,38.458225,41.898029,37.940521,91.326416,10.129892,5.769340,...,3.493703,2.032006,3.054575,2.527580,2.436892,3.938260,102.170700,9.249150,14.327413,3.498808
3,756.789169,10.388915,2.743108,3.358701,469.239627,8.803836,4.070991,76.087097,6.603870,6.868962,...,1.944075,0.275828,0.378870,0.560899,0.876278,0.601052,18.517523,13.592501,27.768326,1.727410
4,2246.720947,33.103138,0.129197,0.102090,16.954773,63.120842,2.530597,269.549683,0.429608,0.043806,...,0.047271,0.067413,0.125030,0.062153,0.037549,0.018097,11.803867,0.048919,12.591056,0.041312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,1122.523928,2.375840,0.117298,0.291367,2.540542,8.341571,7.701284,19.675465,0.384054,0.038858,...,0.189767,0.023321,0.077986,0.114181,0.079088,0.136080,1348.362236,0.031372,0.909478,0.075773
406,1093.928711,30.908592,7.390895,4.472696,380.951721,268.474884,5.851510,462.363556,46.841675,4.414210,...,3.766765,1.473979,2.052898,3.039347,3.245087,1.815966,27.089142,3.699843,139.633942,1.543662
407,11.151926,6.487950,1.008579,0.038034,852.187585,722.604272,0.066098,146.063477,10.495024,0.287135,...,0.025833,0.562429,1.174548,0.054952,0.038135,0.050367,0.548938,0.055491,631.853882,0.009002
408,1000.505249,113.344955,2.114142,36.627228,150.911865,66.287323,3.026417,35.892666,16.383398,402.141968,...,2.154584,3.856749,4.755974,1.392886,1.615846,11.304996,16.827240,380.811279,6.327890,3.683639


In [33]:
# raw = pd.read_csv('D:\\PyCharm\\Py_Projects\\DiTT\\result\\data_hnsc\\original.csv', index_col=0)
raw = adata.X
result_mask = raw
result_mask = np.where(result_mask != 0, 1, 0)
prediction_result_final = raw + impute * (1 - result_mask)
pred_result = pd.DataFrame(prediction_result_final)
raw_data = pd.DataFrame(raw)

pred_result.to_csv('D:/PyCharm/Py_Projects/Diffusion/baseline_result/scVI/scVI_coad_blood_prediction.csv', header=True, index=True)